#  Setup

# New Section

In [0]:
!mkdir -p ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [0]:
!kaggle datasets download -d danofer/dbpedia-classes

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [0]:
!unzip db*.zip

unzip:  cannot find or open db*.zip, db*.zip.zip or db*.zip.ZIP.

No zipfiles found.


In [0]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2019-12-18 09:13:27--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-12-18 09:13:27--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-12-18 09:13:27--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
from functools import wraps
from time import time

def time_wraps(f):
  """A Decorator function to get the time."""
  @wraps(f)
  def wrapper(*args, **kwargs):
    start = time()
    result = f(*args, **kwargs)
    endtime = time()
    print('Elapsed time : {} secs '.format(round((endtime-start), 4)))
    return result
  return wrapper

In [0]:
@time_wraps # - this is a decorator
def read_glove(file_name):
  """A Function to read the glove vectors from file."""
  with open(file_name,'r') as f:
    word_vocab = set() # not using list to avoid duplicate entry
    word2vector = {}
    for line in f:
      line_ = line.strip() #Remove leading and trailing white spaces
      words_Vec = line_.split() #splits the string into list
      word_vocab.add(words_Vec[0]) #Formation of string
      word2vector[words_Vec[0]] = np.array(words_Vec[1:],dtype=float) #formation of array
      pdb.set_trace()
  print("Total Words in DataSet:",len(word_vocab))
  return word_vocab,word2vector

In [0]:
import pandas as pd
import numpy as np

In [0]:
word_vocab, w2v = read_glove('glove.6B.50d.txt')

NameError: ignored

In [0]:
df = pd.read_csv('DBP_wiki_data.csv')
df.head()

FileNotFoundError: ignored

In [0]:
import matplotlib.pyplot as plt

In [0]:
!pip install -q swifter
import swifter

# EDA 1

In [0]:
@time_wraps
def plot_bar(x, y, plt_title):
  """A Function to plot the bar graph."""
  fig, ax = plt.subplots(figsize=(35, 7))
  ax.bar(x, y)
  ax.tick_params('x', rotation=90)
  ax.set_title(plt_title)
  fig.tight_layout()
  plt.show()

In [0]:
# counting the class in dataset
l3_value_counts = df.l3.value_counts()
l3_value_counts.head()

In [0]:
plot_bar(l3_value_counts.index.to_list(), l3_value_counts.to_list(), 'Plot for class counts')

In [0]:
# plotting word_count per class
plot_bar(df.l3.to_list(), df.word_count.to_list(), 'Plot for word count per class')

In [0]:
df.shape

In [0]:
def cal_label_perc(x):
  """A function to calculate the label percentage in the dataset."""
  return (l3_value_counts.loc[x] / df.shape[0]) * 100

In [0]:
df['perc_label'] = df.l3.apply(cal_label_perc) #added a column of calculated label percentage to df

In [0]:
df.head()

In [0]:
for i in range(1, 4):
  new_df = df[df['perc_label'] < (i * 0.25)]
  plot_bar(new_df.l3.to_list(), new_df.perc_label.to_list(), 'classes on {} contribution'.format((i * 0.25)))
  print('--'.join(list(set(new_df.l3.tolist()))))
  del new_df

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
df = df.dropna(axis=0)

In [0]:
df.head()

In [0]:
df.shape

In [0]:
train_data, test_data = train_test_split(df, stratify=df.l3, test_size=0.1, shuffle=True, random_state=42)
train_data.shape, test_data.shape

In [0]:
class_diction = dict((v, i) for i, v in enumerate(list(set(df.l3.to_list())))) #DOUBT
rev_class_diction = dict((i, v) for i, v in zip(class_diction.values(), class_diction.keys()))
len(class_diction)

In [0]:
# change the class function
def class_fn(x):
  """A Function to convert from the class label to number."""
  return class_diction[x]

In [0]:
train_data.l3 = train_data.l3.apply(class_fn)

In [0]:
train_data.head()

**SVC EXP 1** 

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
import collections

In [0]:
class TfidfEmbeddingVectorizer(object):
  def __init__(self, word2vec):
    self.word2vec = word2vec
    self.word2weight = None
    self.dim = 50
  
  def fit(self, X, y):
    tfidf = TfidfVectorizer(analyzer=lambda x : x)
    tfidf.fit(X)
    
    max_idf = max(tfidf.idf_)
    self.word2weight = collections.defaultdict(
        lambda: max_idf,
        [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()]
    )
    return self
  
  def transform(self, X):
    tmp = np.array([
                     [self.word2vec[w] * self.word2weight[w]
                      for w in X if w in self.word2vec] or
                     [np.zeros(self.dim)] for words in X
    ])
    tmp = np.reshape(tmp, (tmp.shape[0], -1))
    return tmp

In [0]:
svc_pipeline = Pipeline([
                         ('word2vec_tfidf', TfidfEmbeddingVectorizer(w2v)),
                         ('svc_clf', SVC(degree=9, kernel='rbf', gamma='scale', probability=True, random_state=42, verbose=True, class_weight='balanced'))
], verbose=True)

In [0]:
epochs = 3

for epoch in range(epochs):
  print('epoch - {}'.format(epoch))
  idx = train_data.sample(n=50000, random_state=42).index
  svc_pipeline.fit(train_data.text.loc[idx], train_data.l3.loc[idx])

In [0]:
from sklearn.metrics import classification_report

In [0]:
# testing the predictions
tmp_idx = test_data.sample(n=50, random_state=42).index
tmp_pred = svc_pipeline.predict_proba(test_data.text.loc[tmp_idx])
tmp_pred

In [0]:
tmp_pred = [np.argmax(i) for i in tmp_pred]
tmp_pred[:10]

In [0]:
y_true = test_data.l3.loc[tmp_idx].apply(class_fn)

In [0]:
print(classification_report(y_true.to_list(), tmp_pred))

# EDA 2

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords
import string
import re
import spacy
from spacy.en import English()

In [0]:
STOP_LIST = set(stopwords.words('english') + ["n't", "'s", "'m", "ca"] + list(ENGLISH_STOP_WORDS))

SYMBOLS = " ".join(string.punctuation).split(" ") + ["-----", "----",  "...", "“", "”", "'ve"]

In [0]:
# tokenizing using spacy
def spacy_tokenize(text):
  """A Functiont to tokenize the text."""
  
  # break into tokens
  tokens = parser(text)
  
  # lemmatize
  lemmas = [tok.lemma_.lower().strip() for tok in tokens if tok.lemma_ != "-PRON-" else tok.lower_]
  tokens = lemmas
  
  # remove stop words
  tokens = [tok for tok in tokens if tok not in STOPLIST]
  
  return tokens

# function to clean the text
def clean_text(text):
  """A Function to clean the bloody text!!"""
  
  # removing spaces and newlines
  text = text.strip().replace("\n", " the text
  spacy_tokens = spacy_tokenize(text)
  
  # replace chinese symbols
  stupid_chinese_symbols = re.findall(ur'[\u4e00-\u9fff]+', text)
  text = " ".join([w for w in spacy_tokens if w not in stupid_chinese_symbols])
  
  # replace html symbols
  text = text.replace("&amp;", "and").replace("&gt;", ">").replace("&lt;", "<" )
  
  # lower the case
  text = text.lower()
  return text

# features with highest coefficient values
def print_highest_coeff(vectorizer, clf, N):
  pass

# custom transformer for cleaning text with spacy
class CleanTextTransformer(TransformerMixin):
  """Clean the bloody text!!"""
  
  def transform(self, X, **transform_params):
    return [clean_text(text) for text in X]
  
  def fit(self, X, y=None, **fit_params):
    return self
  
  def get_params(self, deep=True):
    return {}

In [0]:
# create a vectorizer
vectorizer = CountVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1, 1))

In [0]:
# create the classifier
clf = SVC(probability=True, gamma='scale')

In [0]:
# create a pipeline
pipe = Pipeline([
    ('clean_text', CleanTextTransformer()),
    ('vectorizer', vectorizer),
    ('clf', clf)
])

In [0]:
# fit the model on train data
pipe.fit(train_data.text.to_list(), train_data.l3.to_list())

In [0]:
# get the vectorizer in getting the feature names

vocab = vectorizer.get_feature_names()

# display the transformed vectors
pipe = Pipeline([
    ('clean_text', CleantTextTransformer()),
    ('vectorizer', vectorizer)
])

transform = pipe.fit_transform(train_data.text.to_list(), train_data.l3.to_list())

for i in range(len(train_data)):
  s = ""
  idx_to_vocab = transform.indices[transform.indptr[i] : transform.indptr[i + 1]]
  num_occurences = transform.data[transform.indptr[i] : transform.indptr[i + 1]]
  for idx, num in zip(idx_to_vocab, num_occurences):
    s += str((vocab[idx], num))
  print("Sample {}:{}".format(i, s))


# EDA 3

In [0]:
# remove the sentences with word count less than 5
def word_count_based_removal(df, n):
  """A function to remove the sentences as per the given word count."""
  return df.loc[df[df.word_count > n].index]

# get max min avg of the word count
def get_max_min_avg(df):
  """A Function to get the max min from df."""
  return max(df.word_count), min(df.word_count), np.average(df.word_count)

# Given text count the words
def count_words(text):
  """A function to count the words in the text."""
  return len(text.split())

# remove words as per frequency of the words
def freq_based_removal(df, m, n):
  """A Function to remove as per the frequency."""
  freq = pd.Series(' '.join(df.text).split()).value_counts()
  high_freq = freq[freq > m].index
  low_freq = freq[freq < n].index
  df.text = df.text.swifter.apply(lambda x: ' '.join(x for x in x.split() if x not in high_freq))
  df.text = df.text.swifter.apply(lambda x: ' '.join(x for x in x.split() if x not in low_freq))
  df.word_count = df.text.swifter.apply(count_words)
  return df